In [1]:
from matplotlib import pyplot as plt
import numpy as np
import numba

In [2]:
@numba.guvectorize([numba.void(numba.float64[:,:], numba.float64[:], numba.uint16, numba.uint16[:])], 
                   '(a,b),(b),()->(a)', target='parallel', nopython=True)
def mandelbrot(c,j,maxiter,res):
    for i in range(c.shape[0]):
        creal,cimag,cjmag,ckmag = c[i]
        real, imag, jmag, kmag  = j
        nreal,nimag,njmag,nkmag  = 0, 0, 0, 0
        for n in range(1,maxiter):
            real=nreal*nreal - nimag*nimag - njmag*njmag - nkmag*nkmag + creal
            imag=2*nreal*nimag + cimag
            jmag=2*nreal*njmag + cjmag
            kmag=2*nreal*nkmag + ckmag
            nreal,nimag,njmag,nkmag = real,imag,jmag,kmag
            if real * real + imag * imag + jmag * jmag + kmag * kmag > 4.0:
                res[i] = n
                break
        else:
            res[i] = 0


In [5]:
def plotRijk(rijk, data, axis='ri'):
    swap = np.swapaxes(rijk,0,1)
    dct = {'r':0, 'i':1, 'j':2, 'k':3}
    assert len(axis) in [2,3]
    points = np.array([swap[dct[i]] for i in axis])
    return points

In [4]:
def colourmap(inp):
    @numba.guvectorize([numba.void(numba.int32[:], numba.int32[:], numba.float64[:,:])], 
                       '(a),(b)->(a,b)', target='parallel', nopython=True)
    def colmap(inp, hi, res):
        mx = np.max(inp)
        for i in range(len(inp)):
            if inp[i] != 0:
                res[i] = (1.,1-(inp[i]/mx)**.5,1-(inp[i]/mx)**.5)
            else:
                res[i] = (1,1,1)
                
    return colmap(inp, [0,0,0])

In [6]:
from math import gcd
from itertools import combinations
def isCoprime(*args):
    for i in combinations(args,2):
        if gcd(*i) != 1:
            return False
    return True

In [45]:
@numba.guvectorize([numba.void(numba.float64[:,:], numba.float64[:], numba.float64, numba.uint16, numba.uint16[:])], 
                   '(a,b),(b),(),()->(a)', target='parallel', nopython=True)
def mandelbox(c,j,scale,maxiter,res):
    for i in range(c.shape[0]):
        cx, cy, cz = c[i]
        for n in range(maxiter):
            x,y,z = c[i]
            if x > 1: x = 2 - x
            elif x < -1: x = -2 - x
            if y > 1: y = 2 - y
            elif y < -1: y = -2 - y
            if z > 1: z = 2 - z
            elif z < -1: z = -2 - z
            mag = x*x + y*y + z*z
            if mag < 0.7071:
                x,y,z = 4*x, 4*y, 4*z
            elif mag < 1:
                x,y,z = x/mag, y/mag, z/mag
            elif mag > 4:
                res[i] = n
                break
            x,y,z = scale*x+cx, scale*y+cy, scale*z+cz
        else:
            res[i] = 0

In [37]:
rijk = np.mgrid[-2:2:101j, -2:2:101j, -2:2:101j].reshape(3,-1).T

In [50]:
t = mandelbox(rijk, np.array([0,0,0]), 2, 1000)

In [49]:
all(0 == t)

True